#### Libraries

In [1]:
import pandas as pd
import re
import math

Let's read our new dataset. We will create to separate dataset: on for the EDA and one for machine learning.

In [2]:
data = pd.read_csv('./Data/bt_facial_care_products_with_ings.csv')

In [3]:
data.head()

,Names,Links,Types,Brands,Prices,Capacities,Textures,Types of skin,Launch dates,Replacement for,Price per liter/kilogram/unit,Ingredients
0,Soin Revolumisant Intense Anti-Âge Jour - Revi...,https://www.beaute-test.com/soin-revolumisant-...,cremes_jour,L'Oréal Paris,11.08 €,50 ml,Crème,Toutes,Janvier 2021,Soin Revolumisant Anti-âge Jour - Revitalift F...,358 €/l,"['AQUA', 'GLYCERIN', 'DIMETHICONE', 'ISOHEXADE..."
1,Crème de Jour Anti-Rides Q10 - Cien,https://www.beaute-test.com/day_cream_q10_-_ci...,cremes_jour,Lidl,12.00 €,50 ml,Crème,Toutes,No lauching date provided,Crème de Jour - Cien Beauty,59.80 €/l,"['AQUA', 'DIBUTYL ADIPATE', 'GLYCERIN', 'ETHYL..."
2,BB Crème,https://www.beaute-test.com/soin_miracle_perfe...,cremes_jour,Garnier,5.49 €,50 ml,Crème,Toutes,Septembre 2011,NaN,198 €/l,"['AQUA', 'ISONONYL ISONONANOATE', 'ISOHEXADECA..."
3,BB Skin Detox Fluid SPF 25,https://www.beaute-test.com/bb-skin-detox-flui...,cremes_jour,Clarins,26.90 €,45 ml,Crème,Toutes,No lauching date provided,NaN,833 €/l,"['AQUA', 'DIMETHICONE', 'C10-13 ALKANE', 'CI 7..."
4,Soin Global Anti-Rides Jour - Lift+ Algo Rétinol,https://www.beaute-test.com/soin-global-anti-r...,cremes_jour,Diadermine,12.05 €,50 ml,Crème,Toutes,Janvier 2021,NaN,246 €/l,"['AQUA', 'CAPRYLIC/CAPRIC TRIGLYCERIDE', 'GLYC..."


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16862 entries, 0 to 16861
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Names                          16862 non-null  object
 1   Links                          16862 non-null  object
 2   Types                          16862 non-null  object
 3   Brands                         16862 non-null  object
 4   Prices                         16862 non-null  object
 5   Capacities                     16862 non-null  object
 6   Textures                       16862 non-null  object
 7   Types of skin                  16862 non-null  object
 8   Launch dates                   16862 non-null  object
 9   Replacement for                2195 non-null   object
 10  Price per liter/kilogram/unit  16862 non-null  object
 11  Ingredients                    16862 non-null  object
dtypes: object(12)
memory usage: 1.5+ MB


Some products don't have any ingredients provided so we will drop them.

In [5]:
index_ing_not_provided = data.loc[data.Ingredients == 'NON RENSEIGNE'].index.tolist()

In [6]:
data.drop(index_ing_not_provided, inplace=True)

Some products are replacements for older version of the product so we will drop the older version.

In [7]:
data.loc[data['Replacement for'].isna() == False]

,Names,Links,Types,Brands,Prices,Capacities,Textures,Types of skin,Launch dates,Replacement for,Price per liter/kilogram/unit,Ingredients
0,Soin Revolumisant Intense Anti-Âge Jour - Revi...,https://www.beaute-test.com/soin-revolumisant-...,cremes_jour,L'Oréal Paris,11.08 €,50 ml,Crème,Toutes,Janvier 2021,Soin Revolumisant Anti-âge Jour - Revitalift F...,358 €/l,"['AQUA', 'GLYCERIN', 'DIMETHICONE', 'ISOHEXADE..."
1,Crème de Jour Anti-Rides Q10 - Cien,https://www.beaute-test.com/day_cream_q10_-_ci...,cremes_jour,Lidl,12.00 €,50 ml,Crème,Toutes,No lauching date provided,Crème de Jour - Cien Beauty,59.80 €/l,"['AQUA', 'DIBUTYL ADIPATE', 'GLYCERIN', 'ETHYL..."
6,Génifique Crème Activatrice de Jeunesse,https://www.beaute-test.com/genifique-creme-ac...,cremes_jour,Lancôme,45.90 €,50 ml,Crème,Toutes,No lauching date provided,Génifique Crème,1 660 €/l,"['AQUA', 'GLYCERIN', 'HYDROGENATED POLYISOBUTE..."
7,Crème Cachemire Redensifiante - Resveratrol-Lift,https://www.beaute-test.com/creme_cachemire-re...,cremes_jour,Caudalie,38.90 €,50 ml,Crème,Toutes,No lauching date provided,Crème Cachemire Redensifiante - Resveratrol [L...,798 €/l,"['AQUA', 'COCO-CAPRYLATE/CAPRATE', 'CAPRYLIC/C..."
9,Time-Filler Crème Absolue Correction Rides,https://www.beaute-test.com/time-filler-correc...,cremes_jour,Laboratoires Filorga,19.88 €,50 ml,Crème,Toutes,No lauching date provided,Time-Filler,1 198 €/l,"['AQUA', 'EAU', 'GLYCERIN', 'BUTYLENE GLYCOL',..."
...,...,...,...,...,...,...,...,...,...,...,...,...
16789,Elixir de la Mariée,https://www.beaute-test.com/elixir-mariee-la-s...,lotions,La Sultane de Saba,40.00 €,100 ml,Liquide,Toutes,No lauching date provided,Elixir de la Mariée,290 €/l,"['AQUA', 'PARFUM', 'SODIUM BENZOATE', 'POTASSI..."
16803,Enriched Calming Toner - Skin Recovery,https://www.beaute-test.com/enriched-calming-t...,lotions,Paula's Choice,24.00 €,190 ml,Fluide,Sèche,No lauching date provided,Skin Recovery Toner,126 €/l,"['AQUA', 'CYCLOPENTASILOXANE ', 'BUTYLENE GLYC..."
16833,Eau florale Fleurs d'oranger,https://www.beaute-test.com/eau-florale-fleurs...,lotions,Laboratoire du Haut-Ségala,5.90 €,250 ml,Eau,Toutes,No lauching date provided,Eau de Fleurs d'Oranger Biologique,45.20 €/l,"['CITRUS AURANTIUM AMARA FLOWER WATER', 'POTAS..."
16835,Lotion tonique,https://www.beaute-test.com/lotion-tonique-un-...,lotions,Un Monde de Miel,6.50 €,500 ml,Liquide,Toutes,No lauching date provided,Lotion Tonique au Miel,13.00 €/l,"['AQUA', 'GLYCERIN', 'SODIUM CITRATE', 'PROPYL..."


In [8]:
older_products_data = [data.loc[data['Replacement for'].isna() == False]['Replacement for'],
                       data.loc[data['Replacement for'].isna() == False]['Brands']]
older_products_df = pd.concat(older_products_data, axis=1, keys=['Names', 'Brands'])
older_products_df

,Names,Brands
0,Soin Revolumisant Anti-âge Jour - Revitalift F...,L'Oréal Paris
1,Crème de Jour - Cien Beauty,Lidl
6,Génifique Crème,Lancôme
7,Crème Cachemire Redensifiante - Resveratrol [L...,Caudalie
9,Time-Filler,Laboratoires Filorga
...,...,...
16789,Elixir de la Mariée,La Sultane de Saba
16803,Skin Recovery Toner,Paula's Choice
16833,Eau de Fleurs d'Oranger Biologique,Laboratoire du Haut-Ségala
16835,Lotion Tonique au Miel,Un Monde de Miel


In [9]:
old_names = data.loc[data['Replacement for'].isna() == False]['Replacement for'].tolist()
old_brands = data.loc[data['Replacement for'].isna() == False]['Brands'].tolist()
index_older_products=[]
for i in range(len(older_products_df.Names.tolist())):
        index_older_products.append((data.loc[(data.Names == old_names[i]) & (data.Brands == old_brands[i])].index.tolist()))

In [10]:
index_older_products = [idx[0] for idx in index_older_products if idx]
len(index_older_products)

415

In [11]:
data.drop(index_older_products, inplace=True)

In [12]:
#As we won't need the column "Remplacement for" we'ill drop it
data.drop('Replacement for', axis=1, inplace=True)

We want to drop rows with ingredients which are not properly provided

In [13]:
to_drop = [['N', 'O', 'N', 'R', 'E', 'N', 'S', 'E', 'I', 'G', 'N', 'É'],
           ['>', 'A', 'C', 'T', 'I', 'F', 'S'],
           ['100 % PUR', 'SANS CONSERVATEUR', 'FRAIS ET CONCENTRÉ', 'MICROFILTRÉ À 0', '2 MICRONS POUR UNE PROPRETÉ OPTIMALE', 'CONFORME MICROBIOLOGIQUEMENT'],
           ['S', 'A', 'N', 'S', 'P', 'A', 'R', 'A', 'B', 'E', 'N'],
           ['L', 'E', 'S', 'E', 'C', 'R', 'E', 'T', 'D', 'E', 'L', "'", 'E', 'F', 'F', 'I', 'C', 'A', 'C', 'I', 'T', 'E', 'D', 'E', 'S', 'S', 'O', 'I', 'N', 'S', 'D', 'U', 'J', 'A', 'R', 'D', 'I', 'N', 'D', "'", 'E', 'D', 'E', 'N', 'E', 'N', 'S', '?', 'L', "'", 'E', 'X', 'T', 'R', 'A', 'I', 'T', 'P', 'H', 'Y', 'T', 'O', '-', 'N', 'A', 'T', 'I', 'F', '®'],
           ['A', 'C', 'T', 'I', 'F', 'S'],
           ['T', 'E', 'S', 'T', 'É', 'S', 'O', 'U', 'S', 'C', 'O', 'N', 'T', 'R', 'Ô', 'L', 'E', 'D', 'E', 'R', 'M', 'A', 'T', 'O', 'L', 'O', 'G', 'I', 'Q', 'U', 'E'],
           ['P', 'R', 'I', 'N', 'C', 'I', 'P', 'E', 'S', 'A', 'C', 'T', 'I', 'F', 'S'],
           ['T', 'E', 'S', 'T', 'É', 'D', 'E', 'R', 'M', 'A', 'T', 'O', 'L', 'O', 'G', 'I', 'Q', 'U', 'E', 'M', 'E', 'N', 'T'],
           ['H', 'U', 'I', 'L', 'E', 'S', 'E', 'S', 'S', 'E', 'N', 'T', 'I', 'E', 'L', 'L', 'E', 'S', 'D', 'E', 'R', 'O', 'S', 'E', 'E', 'T', 'D', 'E', 'B', 'A', 'Y', 'S', 'A', 'I', 'N', 'T', 'T', 'H', 'O', 'M', 'A', 'S', 'A', 'V', 'E', 'C', 'D', 'E', 'S', 'H', 'U', 'I', 'L', 'E', 'S', 'D', 'E', 'R', 'O', 'S', 'I', 'E', 'R', 'M', 'U', 'S', 'C', 'A', 'T', 'E', 'T', 'D', "'", 'O', 'N', 'A', 'G', 'R', 'E'],
           ['C', 'A', 'R', 'M', 'E', 'X'],
           ['S', 'A', 'N', 'S', 'P', 'A', 'R', 'F', 'U', 'M'],
           ['D', 'I', 'S', 'T', 'I', 'L', 'L', 'A', 'T', 'I', 'O', 'N', 'P', 'A', 'R', 'E', 'N', 'T', 'R', 'A', 'Î', 'N', 'E', 'M', 'E', 'N', 'T', 'À', 'L', 'A', 'V', 'A', 'P', 'E', 'U', 'R', 'D', "'", 'E', 'A', 'U'],
           ['E', 'A', 'U', 'T', 'H', 'E', 'R', 'M', 'A', 'L', 'E', 'D', "'", 'U', 'R', 'I', 'A', 'G', 'E'],
           ['L', 'E', 'S', 'P', 'R', 'O', 'D', 'U', 'I', 'T', 'S', 'Z', 'A', 'R', 'Q', 'A', 'N', 'E', 'C', 'O', 'N', 'T', 'I', 'E', 'N', 'N', 'E', 'N', 'T', 'P', 'A', 'S', 'D', 'E', ':'],
           ['P', 'R', 'I', 'N', 'C', 'I', 'P', 'A', 'U', 'X', 'C', 'O', 'M', 'P', 'O', 'S', 'A', 'N', 'T', 'S'],
           ['S', 'A', 'N', 'S', 'P', 'A', 'R', 'A', 'B', 'È', 'N', 'E']]

In [14]:
#Change to BUTYROSPERMUM PARKII BUTTER
data.Ingredients = data.Ingredients.replace("['B', 'U', 'T', 'Y', 'R', 'O', 'S', 'P', 'E', 'R', 'M', 'U', 'M', 'P', 'A', 'R', 'K', 'I', 'I']",
                                           "['BUTYROSPERMUM PARKII BUTTER']")

In [15]:
idx_to_drop =[]
for i in range(len(to_drop)):
        idx_to_drop.append((data.loc[data.Ingredients == str(to_drop[i])].index.tolist()))

In [16]:
idx_to_drop = [idx for sublist in idx_to_drop for idx in sublist]

In [17]:
data.drop(idx_to_drop, inplace=True)

Let's drop the rows with no price provided

In [18]:
no_price_idx = data.loc[(data['Price per liter/kilogram/unit'] == 'Non renseigné') | (data.Prices == 'nc €')].index.tolist()

In [19]:
data.drop(no_price_idx, inplace=True)

We want to match the ingredients with the inci dataframe so that we can know if there is any harmful ingredients in the product.

In [20]:
inci_df = pd.read_csv('./Data/incibeauty_ingredients.csv')

In [21]:
inci_df.head()

,Names,Links,Effect,Usages
0,A-TERPINYL ACETATE,https://incibeauty.com/ingredients/18085-a-ter...,vert,['Agent parfumant']
1,ABALONE EXTRACT,https://incibeauty.com/ingredients/8318-abalon...,vert,"[""Agent d'entretien de la peau""]"
2,ABELMOSCHUS ESCULENTUS FRUIT EXTRACT,https://incibeauty.com/ingredients/10626-abelm...,vert,"[""Agent d'entretien de la peau""]"
3,ABIES ALBA CONE OIL,https://incibeauty.com/ingredients/7203-abies-...,jaune,['Agent parfumant']
4,ABIES ALBA EXTRACT,https://incibeauty.com/ingredients/6800-abies-...,vert,"[""Agent d'entretien de la peau""]"


In [22]:
inci_ings_correct = inci_df.loc[inci_df.Effect == 'orange']['Names'].tolist()
inci_ings_not_great = inci_df.loc[inci_df.Effect == 'jaune']['Names'].tolist()
inci_ings_controversial = inci_df.loc[inci_df.Effect == 'rouge']['Names'].tolist()

We're going to create a function which return the number of ingredients which match a list

In [23]:
def number_of_ings_correct_matched(list_a):
    for ing in inci_ings_correct:
        if ing in list_a:
            break
        return len([list_a.index(ing) for ing in inci_ings_correct if ing in list_a])
    else:
        return 0

In [24]:
def number_of_ings_not_great_matched(list_a):
    for ing in inci_ings_correct:
        if ing in list_a:
            break
        return len([list_a.index(ing) for ing in inci_ings_not_great if ing in list_a])
    else:
        return 0

In [25]:
def number_of_ings_controversial_matched(list_a):
    for ing in inci_ings_correct:
        if ing in list_a:
            break
        return len([list_a.index(ing) for ing in inci_ings_controversial if ing in list_a])
    else:
        return 0

In [26]:
data['Number of Correct Ingredients'] = data.Ingredients.apply(number_of_ings_correct_matched)

In [27]:
data['Number of Not Great Ingredients'] = data.Ingredients.apply(number_of_ings_not_great_matched)
data['Number of Controversial Ingredients'] = data.Ingredients.apply(number_of_ings_controversial_matched)

Let's convert the price columns into float

In [28]:
def strip_non_numeric_characters(string_with_numbers):
    return re.sub(r'[^\d.]+', '', string_with_numbers)

In [29]:
data.Prices = data.Prices.apply(strip_non_numeric_characters).astype(float, errors = 'raise')

In [30]:
data['Price per liter/kilogram/unit'] = data['Price per liter/kilogram/unit'].apply(strip_non_numeric_characters).astype(float, errors = 'raise')

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10585 entries, 0 to 16860
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Names                                10585 non-null  object 
 1   Links                                10585 non-null  object 
 2   Types                                10585 non-null  object 
 3   Brands                               10585 non-null  object 
 4   Prices                               10585 non-null  float64
 5   Capacities                           10585 non-null  object 
 6   Textures                             10585 non-null  object 
 7   Types of skin                        10585 non-null  object 
 8   Launch dates                         10585 non-null  object 
 9   Price per liter/kilogram/unit        10585 non-null  float64
 10  Ingredients                          10585 non-null  object 
 11  Number of Correct Ingredient

In [32]:
#Let's put replace the null values in the columns 11 to 13
data['Number of Correct Ingredients'] = data['Number of Correct Ingredients'].fillna(0)
data['Number of Not Great Ingredients'] = data['Number of Not Great Ingredients'].fillna(0)
data['Number of Controversial Ingredients'] = data['Number of Controversial Ingredients'].fillna(0)

In [33]:
data.describe()

,Prices,Price per liter/kilogram/unit,Number of Correct Ingredients,Number of Not Great Ingredients,Number of Controversial Ingredients
count,10585.000000,10585.000000,10585.000000,10585.000000,10585.000000
mean,30.930322,1033.198261,4.307700,6.216438,0.655645
std,36.052912,1763.497136,5.416957,4.531167,1.284826
min,0.450000,0.020000,0.000000,0.000000,0.000000
25%,10.800000,159.000000,0.000000,3.000000,0.000000
50%,21.000000,527.000000,2.000000,6.000000,0.000000
75%,38.900000,1197.000000,7.000000,9.000000,1.000000
max,632.000000,50000.000000,37.000000,27.000000,11.000000


In [34]:
data.loc[data['Price per liter/kilogram/unit']==50000]

,Names,Links,Types,Brands,Prices,Capacities,Textures,Types of skin,Launch dates,Price per liter/kilogram/unit,Ingredients,Number of Correct Ingredients,Number of Not Great Ingredients,Number of Controversial Ingredients
3907,Crème Ultra-Hydratante Invisible - Waso,https://www.beaute-test.com/creme-ultra-hydrat...,cremes_jour_nuit,Shiseido,24.67,1 ml,Crème,Toutes,Août 2017,50000.0,"['WATER', 'GLYCERIN', 'CYCLOPENTASILOXANE', 'D...",22.0,16.0,4.0


In [35]:
#The data for this cream isn't right, so we will rectify it.
data.loc[data['Price per liter/kilogram/unit']==50000,'Prices'] = 30.80
data.loc[data['Price per liter/kilogram/unit']==50000,'Capacities'] = '50 ml'
data.loc[data['Price per liter/kilogram/unit']==50000,'Price per liter/kilogram/unit'] = 616

In [36]:
data.loc[data.Capacities=='1 ml']

,Names,Links,Types,Brands,Prices,Capacities,Textures,Types of skin,Launch dates,Price per liter/kilogram/unit,Ingredients,Number of Correct Ingredients,Number of Not Great Ingredients,Number of Controversial Ingredients
10492,Masque Biocell Hydra+Nourrissant,https://www.beaute-test.com/masque-biocell-hyd...,masques_de_beaute,Science et Mer,9.9,1 ml,Autre,Toutes,No lauching date provided,9900.0,"['AQUA', 'GLYCERIN', 'POLYACRYLATE-13', 'PHENO...",10.0,5.0,1.0


In [37]:
#As product from Science et Mer doesnt' seem to be sold anymore (at least not on the internet),
#we will drop all products from this brand
data.drop(data.loc[data.Brands == 'Science et Mer'].index.tolist(), inplace=True)

Let's change the types of columns Types, Textures, Types of skin

In [38]:
data.Types = data.Types.astype('category')
data.Textures = data.Textures.astype('category')
data['Types of skin'] = data['Types of skin'].astype('category')

In [39]:
#Let's assign the encoded variable to a new column
data['Types_cat'] = data.Types.cat.codes
data['Textures_cat'] = data.Textures.cat.codes
data['Types_of_skin_cat'] = data['Types of skin'].cat.codes

In [40]:
data.head()

,Names,Links,Types,Brands,Prices,Capacities,Textures,Types of skin,Launch dates,Price per liter/kilogram/unit,Ingredients,Number of Correct Ingredients,Number of Not Great Ingredients,Number of Controversial Ingredients,Types_cat,Textures_cat,Types_of_skin_cat
0,Soin Revolumisant Intense Anti-Âge Jour - Revi...,https://www.beaute-test.com/soin-revolumisant-...,cremes_jour,L'Oréal Paris,11.08,50 ml,Crème,Toutes,Janvier 2021,358.0,"['AQUA', 'GLYCERIN', 'DIMETHICONE', 'ISOHEXADE...",19.0,13.0,0.0,2,3,7
1,Crème de Jour Anti-Rides Q10 - Cien,https://www.beaute-test.com/day_cream_q10_-_ci...,cremes_jour,Lidl,12.00,50 ml,Crème,Toutes,No lauching date provided,59.8,"['AQUA', 'DIBUTYL ADIPATE', 'GLYCERIN', 'ETHYL...",8.0,17.0,1.0,2,3,7
2,BB Crème,https://www.beaute-test.com/soin_miracle_perfe...,cremes_jour,Garnier,5.49,50 ml,Crème,Toutes,Septembre 2011,198.0,"['AQUA', 'ISONONYL ISONONANOATE', 'ISOHEXADECA...",16.0,15.0,3.0,2,3,7
3,BB Skin Detox Fluid SPF 25,https://www.beaute-test.com/bb-skin-detox-flui...,cremes_jour,Clarins,26.90,45 ml,Crème,Toutes,No lauching date provided,833.0,"['AQUA', 'DIMETHICONE', 'C10-13 ALKANE', 'CI 7...",12.0,17.0,1.0,2,3,7
4,Soin Global Anti-Rides Jour - Lift+ Algo Rétinol,https://www.beaute-test.com/soin-global-anti-r...,cremes_jour,Diadermine,12.05,50 ml,Crème,Toutes,Janvier 2021,246.0,"['AQUA', 'CAPRYLIC/CAPRIC TRIGLYCERIDE', 'GLYC...",1.0,7.0,0.0,2,3,7


There is too many missing launch dates, it's better to drop the column.

In [41]:
data['Launch dates'].value_counts()

No lauching date provided    9950
Septembre 2020                 32
Janvier 2021                   24
Octobre 2014                   21
Septembre 2016                 17
                             ... 
Mai 2010                        1
Mars 2019                       1
Juillet 2009                    1
Octobre 2010                    1
Juillet 2014                    1
Name: Launch dates, Length: 128, dtype: int64

In [42]:
data.drop('Launch dates', axis=1, inplace=True)

In [43]:
data.to_csv('./Data/bt_facial_care_products_final.csv', index=False)

The majority ingredients on the INCI list of a product (those with a concentration greater than 1%) must be mentioned in order of decreasing concentration. Below 1%, the "minority" ingredients may be indicated out of order. It is estimated that the concentration of an ingredient falls below the threshold of 1% when it appears in fourth, fifth or sixth place on the list, as the case may be.

We will make another column with the first 6 ingredients of the product.

In [44]:
#In the dataframe the ingredients is presented as a string representation of a list, we need to convert it
def convert_string_list_to_list(string_list):
    new_list = string_list.strip('][').split(', ')
    return [element.strip("'").strip('"').strip() for element in new_list if element.strip("'").strip()]

In [45]:
inci_df = pd.read_csv('./Data/incibeauty_ingredients.csv')
inci_list_of_ing = inci_df['Names'].tolist()

In [46]:
def first_5_ingredients(ingredients):
    ing_list = convert_string_list_to_list(ingredients)
    if len(ing_list)>6:
        if False not in [False for ele in ing_list[:6] if ele not in inci_list_of_ing]:
            return ing_list[:6]
        else:
            return 'To delete'
    else:
        if False not in [False for ele in ing_list if ele not in inci_list_of_ing] :
            return ing_list
        else:
            return 'To delete'

In [47]:
data['Majority ingredients'] = data.Ingredients.apply(first_5_ingredients)

We're going to drop all the rows with ingredients which we can't match with inci ingredients. 

In [48]:
data.drop(data.loc[data['Majority ingredients'] == 'To delete'].index.tolist(), axis=0, inplace=True)

In [49]:
data.shape

(5885, 17)

In [50]:
data = data.reset_index().drop('index', axis=1)

In [54]:
#Let's drop the categorical columns
data_ML = data.drop(['Types_cat', 'Textures_cat', 'Types_of_skin_cat'], axis=1)

We now want to reduce to ingredients functions so for that we first need to do the one hot encoding, to have the proper columns. Then we will associatethe weight to each functions.

In [55]:
def functions_product(list_ingredients):
    return [inci_df.loc[inci_df.Names == ing].Usages.tolist()[0] for ing in list_ingredients]

In [56]:
def flatten_list(list_of_sublists):
    return [ele for sublist in list_of_sublists for ele in sublist]

In [57]:
inci_df.Usages = inci_df.Usages.apply(convert_string_list_to_list)

In [58]:
data_ML['Functions'] = data_ML['Majority ingredients'].apply(functions_product)

In [59]:
data_ML['Functions_V2'] = data_ML.Functions.apply(flatten_list)

We will now add which type of functions each ingredient has so that we can factorize on the functions

In [60]:
#We first get the list of unique functions
ings_functions = list(set(flatten_list(inci_df.Usages.tolist())))
len(ings_functions)

75

In [61]:
#We're creating the columns with the functions of each majority ingredients of each product
text_col_functions = [[w for w in row if w in ings_functions] for row in data_ML['Functions_V2']]

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=lambda x:x,
                     preprocessor=lambda x:x)

bow_functions = cv.fit_transform(text_col_functions)
bow_functions.toarray()
bow_functions_df = pd.DataFrame(bow_functions.toarray(), columns=cv.get_feature_names())

bow = cv.fit_transform(text_col_functions)
bow.toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [2, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [63]:
data_ML = pd.concat([data_ML, bow_functions_df], axis=1)

In [64]:
#We have each function from data_ML.columns[1901:]

In [65]:
data_ML.columns[16:]

Index(['Absorbant UV', 'Agent Abrasif', 'Agent Absorbant', 'Agent Oxydant',
       'Agent apaisant', 'Agent arômatisant',
       'Agent bouclant ou lissant (coiffant)', 'Agent colorant pour cheveux',
       'Agent d'entretien de la peau', 'Agent d'entretien de la peau - Divers',
       'Agent d'entretien de la peau - Humectant',
       'Agent d'entretien de la peau - Occlusif', 'Agent d'hygiène buccale',
       'Agent de chélation', 'Agent de contrôle de la viscosité',
       'Agent de fixation capillaire', 'Agent de foisonnement',
       'Agent de protection de la peau', 'Agent de restauration lipidique',
       'Agent démêlant', 'Agent filmogène', 'Agent fixant', 'Agent lissant',
       'Agent masquant', 'Agent moussant', 'Agent nacrant', 'Agent nettoyant',
       'Agent parfumant', 'Agent plastifiant', 'Agent propulseur',
       'Agent rafraîchissant', 'Agent réducteur', 'Agent stabilisant',
       'Agent éclaircissant', 'Agent émulsifiant', 'Ajusteurs de pH',
       'Anti Aggloméra

In [66]:
data_ML[data_ML.columns[16:]] = data_ML[data_ML.columns[16:]].astype(float)

In [67]:
for idx in range(len(data_ML)):
    weight = 0
    for function in list(set(data_ML['Functions_V2'][idx])):
        for i in range(len(data_ML['Functions'][idx])):
            if function in data_ML['Functions'][idx][i]:
                weight = weight + (1/math.sqrt(i+1))
        data_ML.at[idx, function] = weight

In [68]:
data_ML.head()

,Names,Links,Types,Brands,Prices,Capacities,Textures,Types of skin,Price per liter/kilogram/unit,Ingredients,...,Modificateur de surface,Modificateurs de glissement,Non classé,Opacifiant,Régulateur de pH,Sinergiste de mousse,Solvant,Stabilisateur d'émulsion,Tensioactif,Tonifiant
0,Soin Revolumisant Intense Anti-Âge Jour - Revi...,https://www.beaute-test.com/soin-revolumisant-...,cremes_jour,L'Oréal Paris,11.08,50 ml,Crème,Toutes,358.0,"['AQUA', 'GLYCERIN', 'DIMETHICONE', 'ISOHEXADE...",...,0.0,0.0,0.0,11.956251,0.408248,0.000000,7.992910,0.000000,0.000000,0.0
1,Crème de Jour Anti-Rides Q10 - Cien,https://www.beaute-test.com/day_cream_q10_-_ci...,cremes_jour,Lidl,12.00,50 ml,Crème,Toutes,59.8,"['AQUA', 'DIBUTYL ADIPATE', 'GLYCERIN', 'ETHYL...",...,0.0,0.0,0.0,7.641645,0.000000,9.334351,6.079077,10.319949,13.945467,0.0
2,BB Crème,https://www.beaute-test.com/soin_miracle_perfe...,cremes_jour,Garnier,5.49,50 ml,Crème,Toutes,198.0,"['AQUA', 'ISONONYL ISONONANOATE', 'ISOHEXADECA...",...,0.0,0.0,0.0,0.000000,0.000000,0.000000,7.371590,0.000000,0.000000,0.0
3,BB Skin Detox Fluid SPF 25,https://www.beaute-test.com/bb-skin-detox-flui...,cremes_jour,Clarins,26.90,45 ml,Crème,Toutes,833.0,"['AQUA', 'DIMETHICONE', 'C10-13 ALKANE', 'CI 7...",...,0.0,0.0,0.0,6.356808,0.000000,0.000000,5.202488,0.000000,0.000000,0.0
4,Soin Global Anti-Rides Jour - Lift+ Algo Rétinol,https://www.beaute-test.com/soin-global-anti-r...,cremes_jour,Diadermine,12.05,50 ml,Crème,Toutes,246.0,"['AQUA', 'CAPRYLIC/CAPRIC TRIGLYCERIDE', 'GLYC...",...,0.0,0.0,0.0,0.000000,0.000000,0.000000,5.800965,8.926482,0.000000,0.0


In [69]:
data_ML.to_csv('./Data/bt_facial_care_products_for_ML.csv', index=False)